In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rc
rc('font', family = 'Malgun Gothic')
rc('axes', unicode_minus = False)

import time

: 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
from selenium.common.exceptions import NoSuchElementException



: 

In [ ]:
driver =  webdriver.Chrome('./driver/chromedriver.exe')
driver.get('https://www.musinsa.com/categories/item/005?d_cat_cd=005&brand=&list_kind=big&sort=pop_category&sub_sort=&page=2&display_cnt=90&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&plusDeliveryYn=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure=')

ul_element = driver.find_element(By.CSS_SELECTOR, '#searchList') 
li_elements = ul_element.find_elements(By.CSS_SELECTOR, 'li')  

data_no_list = []
for li in li_elements:
    data_no = li.get_attribute('data-no') 
    if data_no != None:
        data_no_list.append(data_no)

driver.quit()

: 

In [ ]:
all_df = pd.DataFrame()
btn_list = ['#estimate_style', '#estimate_photo', '#estimate_goods']

for data_no in data_no_list:
    try:
        driver =  webdriver.Chrome('./driver/chromedriver.exe')
        driver.get('https://www.musinsa.com/app/goods/'+data_no)
        driver.fullscreen_window()
        
        category = driver.find_element(By.CSS_SELECTOR, '#root > div.product-detail__sc-8631sn-0.gJskhq > div.product-detail__sc-8631sn-2.bIfCpH > div.product-detail__sc-up77yl-0.KuCLD > a:nth-child(3)').text
        sex = driver.find_element(By.CSS_SELECTOR, '#root > div.product-detail__sc-8631sn-0.gJskhq > div.product-detail__sc-8631sn-1.fPAiGD > div.product-detail__sc-8631sn-4.goIKhx > div.product-detail__sc-achptn-0.bHXxTQ > ul > li:nth-child(2) > div.product-detail__sc-achptn-6.TehCn > span:nth-child(3)').text
        price = driver.find_element(By.CSS_SELECTOR, '#root > div.product-detail__sc-8631sn-0.gJskhq > div.product-detail__sc-8631sn-1.fPAiGD > div.product-detail__sc-8631sn-4.goIKhx > div.product-detail__sc-w5wkld-0.hgCYZm > div.product-detail__sc-1p1ulhg-0.jEclp > ul > li:nth-child(1) > div.product-detail__sc-1p1ulhg-6.fKNtEN > span').text
        price = price.replace(',','').replace('원','').replace(' ','')

        for btn in btn_list:
            review_btn = driver.find_element(By.CSS_SELECTOR, btn)
            review_btn.click()
            time.sleep(3)

            texts = driver.find_element(By.CSS_SELECTOR, '#reviewListFragment > div.nslist_bottom > div.box_page_msg')
            text = texts.text
            page_str = text[0:text.find('페')]
            page_str = page_str.replace(' ','')
            page_cnt = int(page_str)
            print(f'max page {btn} - {page_cnt}')

            data = []

            for num in range(page_cnt):

                print('current page:',num+1)
                n = num % 5 
                page_btn_lists = driver.find_element(By.CSS_SELECTOR, '#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div') 
                page_btn = page_btn_lists.find_elements(By.CSS_SELECTOR, 'a') 
                page_btn[n+2].click()
                time.sleep(3)

                review_lists = driver.find_element(By.CSS_SELECTOR, '#reviewListFragment') 
                review_list = review_lists.find_elements(By.CSS_SELECTOR, 'div.review-list')  

                for review in review_list:
                    name = review.find_element(By.CSS_SELECTOR, "div.review-profile > div > div.review-profile__text > p.review-profile__name").text
                    date = review.find_element(By.CSS_SELECTOR, "div.review-profile > div > div.review-profile__text > p.review-profile__date").text
                    product = review.find_element(By.CSS_SELECTOR, "div.review-goods-information > div.review-goods-information__item > a").text
                    option = review.find_element(By.CSS_SELECTOR, "div.review-goods-information > div.review-goods-information__item > p > span").text
                    score_str = review.find_element(By.CSS_SELECTOR, "div.review-list__rating-wrap > span > span > span").get_attribute('style')
                    score = int(re.search(r'width:\s*(\d+)%;', score_str).group(1))/20
                    review_text = review.find_element(By.CSS_SELECTOR, "div.review-contents > div.review-contents__text").text
                    help_cnt = review.find_element(By.CSS_SELECTOR, "div.review-evaluation-button--type3 > ul > li:nth-child(1) > label > span").text

                    if btn == '#estimate_style': 
                        style_cnt = review.find_element(By.CSS_SELECTOR, "div.review-evaluation-button--type3 > ul > li:nth-child(2) > label > span").text
                        review_type = 'style'
                    elif btn == '#estimate_photo':
                        style_cnt =  None
                        review_type = 'product'
                    else:
                        style_cnt =  None
                        review_type = 'general '
                    
                    #reviewListFragment > div:nth-child(1) > div.review-contents > div.review-evaluation--type3 > ul > li:nth-child(1)
                    tag_lists = review.find_element(By.CSS_SELECTOR, "div.review-contents > div.review-evaluation--type3 > ul")
                    tag_list = tag_lists.find_elements(By.CSS_SELECTOR, 'li.review-evaluation--type3__item')
                    tags = ''
                    
                    for tag in tag_list:
                        tags = tags + tag.text.replace('\n','-') +', '
                    tags = tags[:len(tags)-2]

                    data.append({
                        "name": name,
                        "date": date,
                        "product": product,
                        "detail": option,
                        "score": score,
                        "review": review_text,
                        "help_cnt": help_cnt,
                        "style_cnt": style_cnt,
                        "type": review_type,
                        "category": category,
                        "sex": sex,
                        "price": price,
                        "tag": tags
                    })

                if num % 5 == 4:
                    next_btn = driver.find_element(By.CSS_SELECTOR,'#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a.fa.fa-angle-right.paging-btn.btn.next')
                    next_btn.click()
                    time.sleep(3)

            df = pd.DataFrame(data)
            all_df = pd.concat([all_df,df])

        driver.quit()
    except NoSuchElementException as e:
        print(f"An error occurred: {e}")
        driver.quit()
        continue

: 

In [ ]:
all_df

: 

In [ ]:
all_df.to_csv('./test2.csv', encoding='euc-kr')

: 